In [34]:

import pandas as pd
import numpy as np

In [35]:
data=pd.read_csv(r".\data\gemstone.csv")
data.drop(labels=["id"],axis=1,inplace=True)
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619.0
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387.0
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772.0
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666.0
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453.0


In [36]:
X=data.drop(labels=["price"],axis=1)
y=data[["price"]]

In [37]:

categorical_cols=X.select_dtypes(include='object').columns

numerical_cols=X.select_dtypes(exclude='object').columns

In [38]:
X.select_dtypes(include='object')

,cut,color,clarity
0,Premium,F,VS2
1,Very Good,J,SI2
2,Ideal,G,VS1
3,Ideal,G,VS1
4,Premium,G,VS2
...,...,...,...
322618,Ideal,D,VVS2
322619,Premium,D,SI1
322620,Premium,G,VVS2
322621,Ideal,D,I1


In [39]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [40]:

from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [41]:
num_pipeline=Pipeline(
    steps=[
        ('imputer', SimpleImputer()),
        ('scaler', StandardScaler())
    ]
)
    

In [42]:
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [43]:
cat_pipeline=Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[ cut_categories
                                                         , color_categories
                                                         , clarity_categories
                                                    ]))
    ]
)

In [44]:
cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good',
                                             'Premium', 'Ideal'],
                                            ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                                            ['I1', 'SI2', 'SI1', 'VS2', 'VS1',
                                             'VVS2', 'VVS1', 'IF']]))])

In [45]:
preprocessor=ColumnTransformer(
    [
        
        ('num_pipeline', num_pipeline, numerical_cols),
        ('cat_pipeline', cat_pipeline, categorical_cols)
    ]
)

In [46]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']]))]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

In [47]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [48]:

preprocessor.fit_transform(X_train)

array([[ 0.77991991,  0.63007597,  0.40285309, ...,  3.        ,
         2.        ,  1.        ],
       [ 2.66232902, -0.38899257,  1.96990468, ...,  2.        ,
         6.        ,  2.        ],
       [ 0.49864038, -0.48163516,  0.40285309, ...,  3.        ,
         0.        ,  1.        ],
       ...,
       [ 0.58518793,  0.07422041,  0.40285309, ...,  2.        ,
         3.        ,  4.        ],
       [ 0.99628877,  0.166863  , -0.64184796, ...,  4.        ,
         3.        ,  1.        ],
       [ 0.67173547, -1.77863149,  1.44755415, ...,  4.        ,
         3.        ,  4.        ]])

In [49]:

preprocessor.transform(X_test)

array([[ 9.91995615e-04, -8.52205544e-01,  9.25203622e-01, ...,
         3.00000000e+00,  4.00000000e+00,  2.00000000e+00],
       [ 2.83542411e+00,  1.66863000e-01,  4.02853095e-01, ...,
         3.00000000e+00,  5.00000000e+00,  3.00000000e+00],
       [ 1.66703225e+00, -5.74277759e-01,  4.02853095e-01, ...,
         3.00000000e+00,  5.00000000e+00,  2.00000000e+00],
       ...,
       [-6.91388369e-01, -1.40806111e+00,  2.49225520e+00, ...,
         2.00000000e+00,  1.00000000e+00,  4.00000000e+00],
       [ 8.66467451e-01, -6.66920354e-01, -1.19497432e-01, ...,
         4.00000000e+00,  4.00000000e+00,  4.00000000e+00],
       [ 7.15009247e-01, -8.52205544e-01,  4.02853095e-01, ...,
         3.00000000e+00,  3.00000000e+00,  1.00000000e+00]])

In [50]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [51]:

X_train=pd.DataFrame(preprocessor.fit_transform(X_train)
                     , columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test)
                    , columns=preprocessor.get_feature_names_out())

In [52]:

"""linear regression
ridge regression
lasso regression
elastic net"""

'linear regression\nridge regression\nlasso regression\nelastic net'

In [53]:
## Model Training

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [54]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [55]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [56]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [57]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    # print(model)

In [58]:
models

{'LinearRegression': LinearRegression(),
 'Lasso': Lasso(),
 'Ridge': Ridge(),
 'Elasticnet': ElasticNet()}

In [59]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test, y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R2 score", r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

ValueError: Input y contains NaN.